In [1]:
import qutip as qt
import numpy as np
print(qt.__version__)



5.0.4


## First we define a random 3 qubit state and its density matrix

In [3]:
def create_random_state(n):
    basis_states = [qt.basis([2]*n, [int(i) for i in format(j, f'0{n}b')]) for j in range(2**n)]
    
    random_probs = np.random.rand(2**n)
    random_probs /= np.sum(random_probs)  # Normalize to sum to 1
    
    coefficients = np.sqrt(random_probs)
   
    random_state = sum(coeff * basis for coeff, basis in zip(coefficients, basis_states))
    return random_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3  

random_state_1 = create_random_state(num_qubits)
random_state_2 = create_random_state(num_qubits)
random_state_3 = create_random_state(num_qubits)

p_random_1 = compute_density_matrix(random_state_1)
p_random_2 = compute_density_matrix(random_state_2)
p_random_3 = compute_density_matrix(random_state_3)

p_mixed = (1/3) * (p_random_1 + p_random_2 + p_random_3)

S_mixed = qt.entropy_vn(p_mixed)

np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:")
print(np.real(p_mixed.full()))
print("\nEntropy of the total system S(A,B,C) =", S_mixed)



Density Matrix in Matrix Form:
[[0.133 0.106 0.115 0.136 0.099 0.122 0.131 0.102]
 [0.106 0.086 0.095 0.117 0.091 0.101 0.105 0.078]
 [0.115 0.095 0.123 0.142 0.108 0.111 0.127 0.075]
 [0.136 0.117 0.142 0.177 0.148 0.139 0.145 0.09 ]
 [0.099 0.091 0.108 0.148 0.14  0.113 0.105 0.062]
 [0.122 0.101 0.111 0.139 0.113 0.118 0.121 0.089]
 [0.131 0.105 0.127 0.145 0.105 0.121 0.138 0.093]
 [0.102 0.078 0.075 0.09  0.062 0.089 0.093 0.085]]

Entropy of the total system S(A,B,C) = 0.3611413068525933


## Then we compute partial traces to get rdms and compute entropies

In [4]:

p_A = p_mixed.ptrace([0])  
p_B = p_mixed.ptrace([1])  
p_C = p_mixed.ptrace([2])  


S_A = qt.entropy_vn(p_A)
S_B = qt.entropy_vn(p_B)
S_C = qt.entropy_vn(p_C)

I_ABC = S_A + S_B + S_C - S_mixed

np.set_printoptions(linewidth=200, precision=3, suppress=True)

print("Reduced Density Matrix ρ_A:\n", np.real(p_A.full()), "\n",f"Von Neumann Entropy S(ρ_A): {S_A:.6f} \n")
print("Reduced Density Matrix ρ_B:\n", np.real(p_B.full()), "\n",f"Von Neumann Entropy S(ρ_B): {S_B:.6f} \n")
print("Reduced Density Matrix ρ_C:\n", np.real(p_C.full()), "\n",f"Von Neumann Entropy S(ρ_C): {S_C:.6f} \n")
print("\n \n Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) = ",I_ABC)

Reduced Density Matrix ρ_A:
 [[0.519 0.417]
 [0.417 0.481]] 
 Von Neumann Entropy S(ρ_A): 0.285835 

Reduced Density Matrix ρ_B:
 [[0.477 0.426]
 [0.426 0.523]] 
 Von Neumann Entropy S(ρ_B): 0.262489 

Reduced Density Matrix ρ_C:
 [[0.534 0.454]
 [0.454 0.466]] 
 Von Neumann Entropy S(ρ_C): 0.184210 


 
 Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) =  0.3713928276138189


## Now we work with some operators correlations, first Z:

In [5]:
Z = qt.sigmaz()

Z_full = qt.tensor(Z, Z, Z)
avg_Z_full = (p_mixed * Z_full).tr()

avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()

Z_norm = Z.norm()

print(f"Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): {avg_Z_full:.6f}")
print(f"Tr_A(p_A * σ_z): {avg_Z_A:.6f}")
print(f"Tr_B(p_B * σ_z): {avg_Z_B:.6f}")
print(f"Tr_C(p_C * σ_z): {avg_Z_C:.6f}")
print(f"Operator norm of σ_z: {Z_norm:.6f}")


Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): 0.132169+0.000000j
Tr_A(p_A * σ_z): 0.038450+0.000000j
Tr_B(p_B * σ_z): -0.045224+0.000000j
Tr_C(p_C * σ_z): 0.067146+0.000000j
Operator norm of σ_z: 2.000000


In [6]:
C_ABC_z = avg_Z_full - avg_Z_A*avg_Z_B*avg_Z_C
f_ABC_z = (C_ABC_z**2)/(2*3*Z_norm**2)

print("Concluding for Z we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_z))

Concluding for Z we have that I(A,B,C) =  0.3713928276138189 While the lower bound is f_ABC =  0.0007291479617473432


## Then X

In [7]:
x = qt.sigmax()

x_full = qt.tensor(x, x, x)
avg_x_full = (p_mixed * x_full).tr()

avg_x_A = (p_A * x).tr()
avg_x_B = (p_B * x).tr()
avg_x_C = (p_C * x).tr()

x_norm = x.norm()

print(f"Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): {avg_x_full:.6f}")
print(f"Tr_A(p_A * σ_x): {avg_x_A:.6f}")
print(f"Tr_B(p_B * σ_x): {avg_x_B:.6f}")
print(f"Tr_C(p_C * σ_x): {avg_x_C:.6f}")
print(f"Operator norm of σ_x: {x_norm:.6f}")

Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): 0.932464+0.000000j
Tr_A(p_A * σ_x): 0.833280+0.000000j
Tr_B(p_B * σ_x): 0.851888+0.000000j
Tr_C(p_C * σ_x): 0.907067+0.000000j
Operator norm of σ_x: 2.000000


In [8]:
C_ABC_x = avg_x_full - avg_x_A*avg_x_B*avg_x_C
f_ABC_x = (C_ABC_x**2)/(2*3*x_norm**2)

print("Concluding for X we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_x))

Concluding for X we have that I(A,B,C) =  0.3713928276138189 While the lower bound is f_ABC =  0.003469748171597082


## And for Y:

In [9]:
y = qt.sigmay()

y_full = qt.tensor(y, y, y)
avg_y_full = (p_mixed * y_full).tr()

avg_y_A = (p_A * y).tr()
avg_y_B = (p_B * y).tr()
avg_y_C = (p_C * y).tr()

y_norm = y.norm()

print(f"Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): {avg_y_full:.6f}")
print(f"Tr_A(p_A * σ_y): {avg_y_A:.6f}")
print(f"Tr_B(p_B * σ_y): {avg_y_B:.6f}")
print(f"Tr_C(p_C * σ_y): {avg_y_C:.6f}")
print(f"Operator norm of σ_y: {y_norm:.6f}")

Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): 0.000000+0.000000j
Tr_A(p_A * σ_y): 0.000000+0.000000j
Tr_B(p_B * σ_y): 0.000000+0.000000j
Tr_C(p_C * σ_y): 0.000000+0.000000j
Operator norm of σ_y: 2.000000


In [10]:
C_ABC_y = avg_y_full - avg_y_A*avg_y_B*avg_y_C
f_ABC_y = (C_ABC_y**2)/(2*3*y_norm**2)

print("Concluding for Y we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_y))

Concluding for Y we have that I(A,B,C) =  0.3713928276138189 While the lower bound is f_ABC =  0.0


## What if I try to mix them?

In [11]:
Mix_operator = qt.tensor(x, Z, Z)
full_mix_avg = (p_mixed * Mix_operator).tr()
C_ABC_MIX = full_mix_avg - avg_x_A*avg_Z_B*avg_Z_C
f_ABC_MIX = (C_ABC_MIX**2)/(2*x_norm**2*Z_norm**2*Z_norm**2)
print("Concluding for a mix of the 3 operators we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_MIX))

Concluding for a mix of the 3 operators we have that I(A,B,C) =  0.3713928276138189 While the lower bound is f_ABC =  4.219649452285904e-05
